# Master Discogs Database

In [52]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils, allmusicUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from artistAM import artistAM
from artistsAM import artistsAM
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-29 20:19:37.567786


In [53]:
%load_ext autoreload
%autoreload
discDC = discogs()
arts = artists(discDC)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available
Warning! Saved Discog Directory None is not Available


# Create Discogs <-> AllMusic Map

In [28]:
discSlimArtistAMDB = getSlimArtistDB(discAM, force=False)
discSlimArtistDCDB = getSlimArtistDB(discDC, force=False)

Current Time is Wed Jan 29, 2020 19:44:40 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/MasterSlimArtistDB.p
  --> This file is 564.1kB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/MasterSlimArtistDB.p
Current Time is Wed Jan 29, 2020 19:44:40 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 32.0 millseconds.
Current Time is Wed Jan 29, 2020 19:44:40 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Loading data from /Users/tgadfort/Music/Discog/db-discogs/MasterSlimArtistDB.p
  --> This file is 57.0MB.
Loading /Users/tgadfort/Music/Discog/db-discogs/MasterSlimArtistDB.p
Current Time is Wed Jan 29, 2020 19:44:42 for Done with 
=================================== Creating Arti

In [31]:
artistDCNameToID = getArtistNameToIDMap(discSlimArtistDCDB)
artistAMNameToID = getArtistNameToIDMap(discSlimArtistAMDB)

Current Time is Wed Jan 29, 2020 19:45:17 for 
=================================== Creating Artist Name To ID Map ===================================
Found 712629 ID -> Name entries
Found 654970 Name -> ID entries
Current Time is Wed Jan 29, 2020 19:45:21 for 
=================================== Creating Artist Name To ID Map ===================================
Found 7559 ID -> Name entries
Found 7365 Name -> ID entries


In [61]:
artistDCKeys = artistDCNameToID.keys()
artistAMKeys = artistAMNameToID.keys()

In [59]:
artistAMToAlbumsData = discAM.getArtistIDToAlbumNamesData(debug=True)
print(len(artistAMToAlbumsData))
artistDCToAlbumsData = discDC.getArtistIDToAlbumNamesData(debug=True)
print(len(artistDCToAlbumsData))

Data stored in /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 3.4MB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
7559
Data stored in /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
712629


In [126]:
from searchUtils import findNearest
num    = 3
cutoff = 0.85

Albums
0000759680 I Wanna Be a Wonderful Milky


In [85]:
flat_list = [item for sublist in l for item in sublist]

[['I Wanna Be a Wonderful Milky']]

In [114]:
x = ['a', 'b', 'cc']
y = 'c'
y in x

False

In [139]:
artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

In [ ]:
#artistAMNameToDCNamesMap = {}
for iKey,artistAMName in enumerate(artistAMKeys):
    if artistAMNametoIDMap.get(artistAMName):
        continue
    if iKey <= 6014:
        continue
    if artistAMName in artistDCKeys:
        continue
        artistAMNameToDCNamesMap[artistAMName] = [artistAMName]
    else:
        artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
        if len(artistDCNames) > 0:
            print(iKey,'/',len(artistAMKeys),'\t',artistAMName)
            artistAMNameToDCNamesMap[artistAMName] = artistDCNames
len(artistAMNameToDCNamesMap)

6019 / 7365 	 Wack Trucks
6038 / 7365 	 Chris Chesney
6046 / 7365 	 Bride of No No
6056 / 7365 	 Brenda Burns
6059 / 7365 	 Anita Valkki
6060 / 7365 	 Cornbred
6064 / 7365 	 Adolphson & Falk
6066 / 7365 	 The Addrisi Brothers
6068 / 7365 	 Church of Betty
6073 / 7365 	 Commodor
6079 / 7365 	 Canzone-Koret
6100 / 7365 	 Crown of Glory
6118 / 7365 	 The Daylight Hours
6119 / 7365 	 Cam Newton
6121 / 7365 	 Allan Davis
6127 / 7365 	 Bob Seger & the Silver Bullet Band
6132 / 7365 	 American Jazz Orchestra
6140 / 7365 	 Larry the Cable Guy
6152 / 7365 	 El Mariachi Vargas de Tecalitlán
6179 / 7365 	 Church of Misery
6185 / 7365 	 Bob the Builder
6187 / 7365 	 Ensemble InterContemporain
6193 / 7365 	 Black Eyed Blonde
6205 / 7365 	 Jam Project
6207 / 7365 	 David M. Patrick
6210 / 7365 	 Benny Goodman & Quartet
6237 / 7365 	 Kjaz
6254 / 7365 	 Barseros
6256 / 7365 	 Cab Calloway & His Cotton Club Orchestra
6262 / 7365 	 Asko | Schönberg
6263 / 7365 	 Ace of Base
6265 / 7365 	 Cannonball Adde

In [138]:
from difflib import SequenceMatcher
import operator

def getArtistAlbums(vals):
    albums = []
    for k,v in vals.items():
        for k2, v2 in v.items():
            albums.append(v2)
    return albums
            
    
def getBestAlbumsMatch(albumsAM, albumsDC, debug=False):
    score = 0.0
    for albumAMAlbum in albumsAM:
        for albumDCAlbum in albumsDC:
            s = SequenceMatcher(None, albumAMAlbum, albumDCAlbum)
            ratio = round(s.ratio(), 3)
            if debug:
                print("\t\t\t[{0}] + [{1}] = [{2}]".format(albumAMAlbum, albumDCAlbum, ratio))
            if ratio > 0.7:
                score += ratio
    return score

def showNames(artistAMName, artistDCNames):
    print("{0: <50}{1}".format(artistAMName, artistDCNames))



artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

for iKey,artistAMName in enumerate(artistAMKeys):
    
    if artistAMNametoIDMap.get(artistAMName):
        continue
    artistDCNames = artistAMNameToDCNamesMap.get(artistAMName)
    if artistDCNames is None:
        continue
    #artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
    #if len(artistDCNames) == 0:
    #    continue
    
    artistAMIDs = artistAMNameToID[artistAMName]
    
    ## Loop over Artist AM IDs
    for iAMID,artistAMID in enumerate(artistAMIDs):
        if artistAMIDtoDCIDMap.get(artistAMID) is not None:
            continue
        
        showNames(artistAMName, artistDCNames)
        artistAMAlbums = getArtistAlbums(artistAMToAlbumsData.get(artistAMID))
        if artistAMAlbums is None:
            print("\tAM ID={0: <15}{1: <5}/{2: <5}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, "NONE"))
        else:
            print("\tAM ID={0: <15}{1: <5}/{2: <5}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, len(artistAMAlbums)))

            
        artistAMtoDCScore = None
        
            
        ## Loop over Artist DC Names
        for artistDCName in artistDCNames:
            artistDCIDs = artistDCNameToID[artistDCName]
            
            ## Loop over Artist DC IDs
            for artistDCID in artistDCIDs:
                artistDCAlbums = getArtistAlbums(artistDCToAlbumsData.get(artistDCID))
                if False:
                    if artistDCAlbums is None:
                        print("\t\tDC ID={0: <15}\t      {1}".format(artistDCID, "NONE"))
                    else:
                        print("\t\tDC ID={0: <15}\t Size={1}".format(artistDCID, len(artistDCAlbums)))
                
                retval = getBestAlbumsMatch(artistAMAlbums, artistDCAlbums, debug=False)
                if retval > 0.7:
                    if artistAMtoDCScore is None:
                        artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                    else:
                        if retval > artistAMtoDCScore["Score"]:
                            artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                            
                            
        print("\tAM ID={0: <15}{1}".format(artistAMID, artistAMtoDCScore))
        if artistAMtoDCScore is not None:
            artistAMIDtoDCIDMap[artistAMID] = {"AM Name": artistAMName, "DC ID": artistAMtoDCScore["ID"], "DC Name": artistAMtoDCScore["Name"]}
            print("Found Match for {0} --> {1}".format(artistAMID, artistAMIDtoDCIDMap[artistAMID]))
            saveFile(idata=artistAMIDtoDCIDMap, ifile="artistAMIDtoDCIDMap.p")
            print("\n")
        


Bambir                                            ['Bambi']
	AM ID=0000396306     4    /7365 0    2
	AM ID=0000396306     None
Alton Chan                                        ['Milton Chan']
	AM ID=0002200106     14   /7365 0    1
	AM ID=0002200106     None
Cherubia                                          ['Cherub']
	AM ID=0001873906     34   /7365 0    1
	AM ID=0001873906     None
Fish On Friday                                    ['No Fish On Friday']
	AM ID=0003068306     38   /7365 0    5
	AM ID=0003068306     None
Berlin RIAS Symphony Orchestra                    ['Berne Symphony Orchestra']
	AM ID=0002268206     42   /7365 0    9
	AM ID=0002268206     None
Southern Rains                                    ['Southern Rail', 'Southern Train', 'Southern Reign']
	AM ID=0003253706     50   /7365 0    1
	AM ID=0003253706     None
Sir Charles Jones                                 ['Charles Jones', 'Sir Charles Groves']
	AM ID=0002059306     52   /7365 0    11
	AM ID=0002059306     Non

	AM ID=0002857769     None
Alex Baird                                        ['Alex Bigard']
	AM ID=0002303969     2179 /7365 0    1
	AM ID=0002303969     None
Colossus of Fall                                  ['Colossus Fall']
	AM ID=0000776504     2222 /7365 0    2
	AM ID=0000776504     None
Above the Orange Trees                            ['Above The Orange Trees']
	AM ID=0001418604     2226 /7365 0    4
	AM ID=0001418604     None
Cows in the Graveyard                             ['Ghosts in the Graveyard']
	AM ID=0001291504     2239 /7365 0    1
	AM ID=0001291504     None
Vienna State Opera Orchestra                      ['Berlin State Opera Orchestra']
	AM ID=0000163304     2252 /7365 0    45
	AM ID=0000163304     None
Bla                                               ['Byla', 'Bula', 'Bola']
	AM ID=0002422596     2283 /7365 0    2
	AM ID=0002422596     None
Deena Jones                                       ['Deena Jonez', 'Deekay Jones', 'Rena Jones']
	AM ID=0001751496     2296 

	AM ID=0001650639     None
GumBoots                                          ['Gumboots']
	AM ID=0001828239     3634 /7365 0    2
	AM ID=0001828239     None
Oceans and Vessels                                ['Oceans And Vessels']
	AM ID=0003457939     3646 /7365 0    1
	AM ID=0003457939     None
Martians                                          ['Martian', 'Marians', 'Martinus']
	AM ID=0001722237     3689 /7365 0    2
	AM ID=0001722237     None
100 Monkeys                                       ['100th Monkey']
	AM ID=0002443137     3694 /7365 0    2
	AM ID=0002443137     None
Big Hilla                                         ['Big Hill']
	AM ID=0001938837     3696 /7365 0    1
	AM ID=0001938837     None
Dee Black                                         ['Deep Black', 'Tee Black', 'D Black']
	AM ID=0001828337     3701 /7365 0    1
	AM ID=0001828337     None
Frankie Mann                                      ['Frankie Moran', 'Frankie May', 'Frankie M']
	AM ID=0001258337     3713 /7365 0 

	AM ID=0000069402     {'Name': 'Bob Marley & The Wailers', 'ID': '21764', 'Score': 362.5610000000005}
Found Match for 0000069402 --> {'AM Name': 'Bob Marley & the Wailers', 'DC ID': '21764', 'DC Name': 'Bob Marley & The Wailers'}
  --> This file is 159.1kB.


Blood on the Dance Floor                          ['Blood On The Dance Floor']
	AM ID=0002574844     4289 /7365 0    15
	AM ID=0002574844     {'Name': 'Blood On The Dance Floor', 'ID': '2126608', 'Score': 7.671}
Found Match for 0002574844 --> {'AM Name': 'Blood on the Dance Floor', 'DC ID': '2126608', 'DC Name': 'Blood On The Dance Floor'}
  --> This file is 159.2kB.


Blessed by a Broken Heart                         ['Blessed By A Broken Heart']
	AM ID=0001005044     4298 /7365 0    5
	AM ID=0001005044     {'Name': 'Blessed By A Broken Heart', 'ID': '1213145', 'Score': 4.633}
Found Match for 0001005044 --> {'AM Name': 'Blessed by a Broken Heart', 'DC ID': '1213145', 'DC Name': 'Blessed By A Broken Heart'}
  --> This file is 159.

  --> This file is 160.1kB.


The Bellamy Brothers                              ['The Kelly Brothers', 'Bellamy Brothers', 'The Bewley Brothers']
	AM ID=0000037026     4473 /7365 0    99
	AM ID=0000037026     {'Name': 'Bellamy Brothers', 'ID': '180931', 'Score': 105.90299999999998}
Found Match for 0000037026 --> {'AM Name': 'The Bellamy Brothers', 'DC ID': '180931', 'DC Name': 'Bellamy Brothers'}
  --> This file is 160.1kB.


DJ Tedu                                           ['DJ Ted']
	AM ID=0000464826     4474 /7365 0    4
	AM ID=0000464826     None
Steve Hobbs                                       ['Steve Cobb']
	AM ID=0000755326     4476 /7365 0    7
	AM ID=0000755326     None
Dawn of Humans                                    ['Dawn Of Humans']
	AM ID=0003390226     4486 /7365 0    1
	AM ID=0003390226     {'Name': 'Dawn Of Humans', 'ID': '2223424', 'Score': 0.929}
Found Match for 0003390226 --> {'AM Name': 'Dawn of Humans', 'DC ID': '2223424', 'DC Name': 'Dawn Of Humans'}
  --> Thi

...

  --> This file is 161.7kB.


The Common                                        ['Thee Commons', 'The Communion']
	AM ID=0000318123     5002 /7365 0    3
	AM ID=0000318123     None
Autoven                                           ['Autovon', 'Autogen']
	AM ID=0003567223     5037 /7365 0    3
	AM ID=0003567223     None
David Becvar                                      ['David Beard']
	AM ID=0003233623     5041 /7365 0    1
	AM ID=0003233623     None
White Jazz Band                                   ['Kite Jazz Band']
	AM ID=0002195023     5048 /7365 0    1
	AM ID=0002195023     None
Dashboard Saviors                                 ['The Dashboard Saviors']
	AM ID=0000141423     5051 /7365 0    3
	AM ID=0000141423     {'Name': 'The Dashboard Saviors', 'ID': '352467', 'Score': 2.933}
Found Match for 0000141423 --> {'AM Name': 'Dashboard Saviors', 'DC ID': '352467', 'DC Name': 'The Dashboard Saviors'}
  --> This file is 161.8kB.


The Original Five Blind Boys of Mississippi       ['The O

  --> This file is 162.5kB.


Jackson Southernaires                             ['The Jackson Southernaires']
	AM ID=0000767967     5247 /7365 0    42
	AM ID=0000767967     {'Name': 'The Jackson Southernaires', 'ID': '1927863', 'Score': 21.665999999999997}
Found Match for 0000767967 --> {'AM Name': 'Jackson Southernaires', 'DC ID': '1927863', 'DC Name': 'The Jackson Southernaires'}
  --> This file is 162.6kB.


Dino Valente                                      ['Dino Valenti', 'Disco Valente']
	AM ID=0000823867     5248 /7365 0    1
	AM ID=0000823867     {'Name': 'Dino Valenti', 'ID': '555687', 'Score': 1.0}
Found Match for 0000823867 --> {'AM Name': 'Dino Valente', 'DC ID': '555687', 'DC Name': 'Dino Valenti'}
  --> This file is 162.6kB.


Surf Raiders                                      ['The Surf Raiders']
	AM ID=0000568067     5251 /7365 0    1
	AM ID=0000568067     None
Bea Arthur                                        ['Jean Arthur']
	AM ID=0000791467     5257 /7365 0    1
	AM I

	AM ID=0000958519     {'Name': 'The Dave Brubeck Octet', 'ID': '1884149', 'Score': 5.41}
Found Match for 0000958519 --> {'AM Name': 'Dave Brubeck Octet', 'DC ID': '1884149', 'DC Name': 'The Dave Brubeck Octet'}
  --> This file is 163.3kB.


Albert Ammons & His Rhythm Kings                  ['Albert Ammons And His Rhythm Kings']
	AM ID=0000933419     5443 /7365 0    2
	AM ID=0000933419     {'Name': 'Albert Ammons And His Rhythm Kings', 'ID': '843843', 'Score': 1.721}
Found Match for 0000933419 --> {'AM Name': 'Albert Ammons & His Rhythm Kings', 'DC ID': '843843', 'DC Name': 'Albert Ammons And His Rhythm Kings'}
  --> This file is 163.4kB.


Björn J:son Lindh                                 ['Björn J:Son Lindh']
	AM ID=0000769719     5445 /7365 0    23
	AM ID=0000769719     {'Name': 'Björn J:Son Lindh', 'ID': '304396', 'Score': 14.922}
Found Match for 0000769719 --> {'AM Name': 'Björn J:son Lindh', 'DC ID': '304396', 'DC Name': 'Björn J:Son Lindh'}
  --> This file is 163.4kB.


Computer 

	AM ID=0000283129     {'Name': 'Rob McConnell & The Boss Brass', 'ID': '931123', 'Score': 31.812}
Found Match for 0000283129 --> {'AM Name': 'Rob McConnell & the Boss Brass', 'DC ID': '931123', 'DC Name': 'Rob McConnell & The Boss Brass'}
  --> This file is 164.2kB.


Kontra Quartet                                    ['Kontras Quartet', 'The Kontra Quartet']
	AM ID=0002196529     5629 /7365 0    14
	AM ID=0002196529     {'Name': 'The Kontra Quartet', 'ID': '1105932', 'Score': 4.578}
Found Match for 0002196529 --> {'AM Name': 'Kontra Quartet', 'DC ID': '1105932', 'DC Name': 'The Kontra Quartet'}
  --> This file is 164.2kB.


The Casualties of Jazz                            ['The Casualties Of Jazz']
	AM ID=0000514529     5630 /7365 0    1
	AM ID=0000514529     None
Dead as Alive                                     ['Dead Dead Alive']
	AM ID=0003528629     5633 /7365 0    4
	AM ID=0003528629     None
Velvet Monkeys                                    ['The Velvet Monkeys']
	AM ID=0000313

	AM ID=0003319871     5888 /7365 0    2
	AM ID=0003319871     None
Cougar Creek                                      ['Sugar Creek']
	AM ID=0002818471     5889 /7365 0    1
	AM ID=0002818471     None
Amusement Parks on Fire                           ['Amusement Parks On Fire']
	AM ID=0000518871     5890 /7365 0    6
	AM ID=0000518871     {'Name': 'Amusement Parks On Fire', 'ID': '339807', 'Score': 3.846}
Found Match for 0000518871 --> {'AM Name': 'Amusement Parks on Fire', 'DC ID': '339807', 'DC Name': 'Amusement Parks On Fire'}
  --> This file is 165.0kB.


Plane                                             ['PlanBe', 'Plan', 'Pane']
	AM ID=0001945871     5903 /7365 0    13
	AM ID=0001945871     None
Caddywampus                                       ['Caddywhompus']
	AM ID=0001707671     5908 /7365 0    1
	AM ID=0001707671     None
Death in June                                     ['Death In June']
	AM ID=0000228371     5910 /7365 0    40
	AM ID=0000228371     {'Name': 'Death In June',

In [129]:
len(artistAMIDtoDCIDMap)

4487

# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### Dictionaries

In [9]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 29, 2020 18:22:46 for Creating Artist DBs
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/6-Metadata.p  	0 71
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/94-Metadata.p  	1 134
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/77-Metadata.p  	2 202
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/40-Metadata.p  	3 270
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/15-Metadata.p  	4 341
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/22-Metadata.p  	5 406
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/88-Metadata.p  	6 481
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/99-Metadata.p  	7 560
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/33-Metadata.p  	8 647
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/51-Metadata.p  	9 707
/Users/tgadfort/Music/Discog/artists-allmusic-db

  --> This file is 191.4kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToRef.p
  --> This file is 191.4kB.
Saving 7559 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
  --> This file is 147.4kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
  --> This file is 147.4kB.
Saving 7559 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 170.9kB.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 170.9kB.
Current Time is Wed Jan 29, 2020 18:22:47 for Done with Creating Artist DBs
Process [{0}] took 1 seconds.


In [22]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Jan 26, 2020 19:26:56 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-Metadata.p  	0 7042
/Volumes/Music/Discog/artists-db/metadata/1-Metadata.p  	1 14333
/Volumes/Music/Discog/artists-db/metadata/10-Metadata.p  	2 21410
/Volumes/Music/Discog/artists-db/metadata/11-Metadata.p  	3 28588
/Volumes/Music/Discog/artists-db/metadata/12-Metadata.p  	4 35774
/Volumes/Music/Discog/artists-db/metadata/13-Metadata.p  	5 43002
/Volumes/Music/Discog/artists-db/metadata/14-Metadata.p  	6 50003
/Volumes/Music/Discog/artists-db/metadata/15-Metadata.p  	7 57131
/Volumes/Music/Discog/artists-db/metadata/16-Metadata.p  	8 64198
/Volumes/Music/Discog/artists-db/metadata/17-Metadata.p  	9 71368
/Volumes/Music/Discog/artists-db/metadata/18-Metadata.p  	10 78533
/Volumes/Music/Discog/artists-db/metadata/19-Metadata.p  	11 85666
/Volumes/Music/Discog/artists-db/metadata/2-Metadata.p 

#### Pandas DB

In [10]:
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()

Current Time is Wed Jan 29, 2020 18:23:53 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
  --> This file is 147.4kB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToName.p
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToRef.p
  --> This file is 191.4kB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToRef.p
Creating Pandas DataFrame for 7559 Artists
	Shape --> (7559, 1)
  Finding Real Artist Name
	Shape --> (7559, 3)
  Removing None Artist
	Shape --> (7559, 3)
  Finding Disc Artist Name
	Shape --> (7559, 4)
DataFrame Shape is (7559, 4)
Current Time is Wed Jan 29, 2020 18:23:53 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 251.0 millseconds.
Saving Master Artist DB File
  --> This file is 564.1kB.


,Name,Artist,Num,DiscArtist
0003743406,Dionysis Behnam,Dionysis Behnam,None,Dionysis Behnam
0001617506,Bebe Rebozo,Bebe Rebozo,None,Bebe Rebozo
0002313106,Blair,Blair,None,Blair
0001417206,Citizen Kain,Citizen Kain,None,Citizen Kain
0000396306,Bambir,Bambir,None,Bambir


In [26]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc, force=True)
discSlimArtistDB.head()

Current Time is Sun Jan 26, 2020 19:38:42 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 712629 Artists
	Shape --> (712629, 1)
  Finding Real Artist Name
	Shape --> (712629, 3)
  Removing None Artist
	Shape --> (712629, 3)
  Finding Disc Artist Name
	Shape --> (712629, 4)
DataFrame Shape is (712629, 4)
Current Time is Sun Jan 26, 2020 19:39:09 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 26 seconds.
Saving Master Artist DB File
  --> This file is 57.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist ID --> Albums

#### Dictionaries

In [11]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 29, 2020 18:24:14 for Creating Artist DBs
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/51-MediaMetadata.p 	60        60        294       
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/6-MediaMetadata.p 	131       131       589       
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/16-MediaMetadata.p 	199       199       909       
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/32-MediaMetadata.p 	287       287       1340      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/94-MediaMetadata.p 	350       350       1698      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/75-MediaMetadata.p 	441       441       2125      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/48-MediaMetadata.p 	527       527       2644      
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/47-MediaMetadata.p 	600   

/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/30-MediaMetadata.p 	5693      5693      29654     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/77-MediaMetadata.p 	5761      5761      30005     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/53-MediaMetadata.p 	5843      5843      30411     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/14-MediaMetadata.p 	5917      5917      30702     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/4-MediaMetadata.p 	5990      5990      31037     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/29-MediaMetadata.p 	6063      6063      31378     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/61-MediaMetadata.p 	6129      6129      31792     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/26-MediaMetadata.p 	6209      6209      32248     
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/80-MediaMetadata.p 	6292      6292      32659     
/Users/tgadfort/Musi

In [27]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Jan 26, 2020 19:51:41 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-MediaMetadata.p 	7042      3616      19379     
/Volumes/Music/Discog/artists-db/metadata/1-MediaMetadata.p 	14333     7313      39865     
/Volumes/Music/Discog/artists-db/metadata/10-MediaMetadata.p 	21410     10958     59764     
/Volumes/Music/Discog/artists-db/metadata/11-MediaMetadata.p 	28588     14632     79734     
/Volumes/Music/Discog/artists-db/metadata/12-MediaMetadata.p 	35774     18344     99718     
/Volumes/Music/Discog/artists-db/metadata/13-MediaMetadata.p 	43002     22014     119840    
/Volumes/Music/Discog/artists-db/metadata/14-MediaMetadata.p 	50003     25628     139333    
/Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p 	57131     29263     159809    
/Volumes/Music/Discog/artists-db/metadata/16-MediaMetadata.p 	64198     32872     179952    
/Volumes/Mu

#### Pandas DB

In [12]:
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

Current Time is Wed Jan 29, 2020 18:24:56 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 3.4MB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 7559 Artists
	Shape --> (7559, 1)
DataFrame Shape is (7559, 1)
Saving Master Artist Albums DB File
  --> This file is 6.8MB.
Current Time is Wed Jan 29, 2020 18:24:58 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 2 seconds.


,Albums
0003821051,"{'Albums': {}, 'Album': {'0003259415': 'No Te ..."
0000076451,{'Albums': {'0000026176': 'One Nation Under a ...
0000667251,{'Albums': {'0000185228': 'Valentine Kid's Lit...
0002933651,"{'Albums': {'0002692462': 'Blues Pills', '0002..."
0001040051,"{'Albums': {}, 'Album': {'0000807741': 'The Se..."


In [28]:
from masterdb import getArtistAlbumsDB
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=True)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 20:09:16 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712629 Artists
	Shape --> (712629, 1)
DataFrame Shape is (712629, 1)
Saving Master Artist Albums DB File
  --> This file is 375.0MB.
Current Time is Sun Jan 26, 2020 20:12:33 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 3.3 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Artist ID --> Genre, Style, Artists Lookup Table

In [13]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 29, 2020 18:25:26 for Creating Artist DBs



Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saving 0 entries to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p

Saving data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Saved data to /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file

In [35]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:27:15 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p 	4771
/Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p 	9481
/Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p 	16165
/Volumes/Music/Discog/albums-db/metadata/11-ArtistMetadata.p 	22936
/Volumes/Music/Discog/albums-db/metadata/12-ArtistMetadata.p 	29738
/Volumes/Music/Discog/albums-db/metadata/13-ArtistMetadata.p 	33792
/Volumes/Music/Discog/albums-db/metadata/14-ArtistMetadata.p 	40028
/Volumes/Music/Discog/albums-db/metadata/15-ArtistMetadata.p 	46750
/Volumes/Music/Discog/albums-db/metadata/16-ArtistMetadata.p 	52022
/Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p 	56703
/Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p 	63415
/Volumes/Music/Discog/albums-db/metadata/19-ArtistMetadata.p 	69777
/Volumes/Music/Discog/albums-db/me

## Album ID --> Name, Ref, Artists Lookup Table

In [36]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:31:37 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p 	21825
/Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p 	41453
/Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p 	67791
/Volumes/Music/Discog/albums-db/metadata/11-ArtistAlbums.p 	89756
/Volumes/Music/Discog/albums-db/metadata/12-ArtistAlbums.p 	116582
/Volumes/Music/Discog/albums-db/metadata/13-ArtistAlbums.p 	129077
/Volumes/Music/Discog/albums-db/metadata/14-ArtistAlbums.p 	152458
/Volumes/Music/Discog/albums-db/metadata/15-ArtistAlbums.p 	174344
/Volumes/Music/Discog/albums-db/metadata/16-ArtistAlbums.p 	192435
/Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p 	206069
/Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p 	230543
/Volumes/Music/Discog/albums-db/metadata/19-ArtistAlbums.p 	250792
/Volumes/Music/Discog/albums-db/metadata/2-Artis

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [13]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

Current Time is Sun Jan 26, 2020 18:43:38 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 712628 Artists
	Shape --> (712628, 1)
  Finding Real Artist Name
	Shape --> (712628, 3)
  Removing None Artist
	Shape --> (712628, 3)
  Finding Disc Artist Name
	Shape --> (712628, 4)
DataFrame Shape is (712628, 4)
Current Time is Sun Jan 26, 2020 18:43:57 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 19 seconds.
Saving Master Artist DB File
  --> This file is 57.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist DB

In [19]:
from masterdb import getArtistDB

In [20]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

Current Time is Wed Jan 22, 2020 20:51:53 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToVariations.p
  --> This file is 26.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToVariations.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
  Joining Ref
	Shape --> (712336, 2)
  Joining Variations
	Shape --> (712336, 3)
  Finding Real Artist Name
	Shape --> (712336, 6)
DataFrame Shape is (712336, 6)
Current Time is Wed Jan 22, 2020 20:52:30 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 36 seconds.
Saving Mas

,Name,Ref,Variations,Known,Artist,Num
1000,Dave Clarke,/artist/1000-Dave-Clarke,"[Dave Clarke, 7 Red 7, Clarck, Clarke, Clarke,...",True,Dave Clarke,NaN
1000500,Club Pulse,/artist/1000500-Club-Pulse,[Club Pulse],True,Club Pulse,NaN
100200,Dike,/artist/100200-Dike,"[Dike, D.I.K.E., Dike D, Uchegdu]",True,Dike,NaN
1002000,Larry Stokes,/artist/1002000-Larry-Stokes,[Larry Stokes],True,Larry Stokes,NaN
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez,[Gidd Sanchez],True,Gidd Sanchez,NaN


## Artist Metadata DB

In [39]:
from masterdb import getArtistMetadataDB

In [40]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

Current Time is Fri Jan 24, 2020 07:20:08 for 
=================================== Creating Artist Metadata DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 12.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 17.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
Creating Pandas DataFrame for 572424 Artists
	Shape --> (572424, 1)
  Joining Style
	Shape --> (572424, 2)
  Joining Collaboration
	Shape --> (572424, 3)
DataFrame Shape is (572424, 3)
Current Time is Fri Jan 24, 2020 07:21:41 for Done with 
=================================== Creating Artist Metadata DB ===================================
Process [{0}] took 1.6 minutes.
Saving Master Artist M

,Genre,Style,Collaboration
1000,"{'Electronic': 55, 'Hip Hop': 1, 'Non-Music': ...","{'Techno': 46, 'Electro': 17, 'Downtempo': 1, ...","{'Dave Clarke': 54, 'Bang The Future': 1, 'Mr...."
1000500,{'Electronic': 1},{'Progressive Trance': 1},{'Club Pulse': 1}
100200,{'Hip Hop': 11},"{'Conscious': 7, 'Pop Rap': 2, '': 3}","{'Dike': 11, 'ABS (2)': 2, 'Creutzfeld&Jakob':..."
1002000,{},{},{}
1005400,{'Rock': 5},"{'Alternative Rock': 1, 'Indie Rock': 1, 'Math...","{'Gapeseed': 5, 'Gerling': 1}"


## Artist Albums DB

In [14]:
from masterdb import getArtistAlbumsDB

In [21]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 19:01:34 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Loading data from /Volumes/Music/Discog/db/MasterArtistAlbumsDB.p
  --> This file is 375.0MB.
Loading /Volumes/Music/Discog/db/MasterArtistAlbumsDB.p


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


In [20]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

Current Time is Sun Jan 26, 2020 18:58:04 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712628 Artists
	Shape --> (712628, 1)
DataFrame Shape is (712628, 1)
Saving Master Artist Albums DB File
  --> This file is 375.0MB.
Current Time is Sun Jan 26, 2020 19:00:52 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 2.8 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Albums DB

In [26]:
from masterdb import getAlbumDB

In [27]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

Current Time is Wed Jan 22, 2020 21:10:30 for 
=================================== Creating Artist Album DB ===================================
Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 39.7MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 55.9MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtists.p
  --> This file is 22.8MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtists.p
Creating Pandas DataFrame for 1695744 Albums
	Shape --> (1695744, 1)
  Joining Ref
	Shape --> (1695744, 2)
  Joining Artists
	Shape --> (1695744, 3)
DataFrame Shape is (1695744, 3)
Current Time is Wed Jan 22, 2020 21:11:41 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 1.2 minutes.


,Name,Ref,Artists
1012077,Live,/Dave-Clarke-Live/master/1012077,[1000]
1258261,Walls Of Genius And Miracle,/Walls-Of-Genius-and-Miracle-Walls-Of-Genius-A...,"[1009227, 182635, 1000]"
2546978,RA.EX007 Dave Clarke,/Dave-Clarke-RAEX007-Dave-Clarke/release/2546978,[1000]
44815,Take Two,/6th-Sense-Approach-Take-Two/release/44815,"[23951, 1000]"
44831,Devil's Advocate,/Dave-Clarke-Devils-Advocate/master/44831,[1000]


## Artist Album ID --> Known Albums

In [28]:
from masterdb import getArtistAlbumKnownDB

In [29]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

Current Time is Wed Jan 22, 2020 21:11:41 for 
=================================== Creating Artist Album DB ===================================
Creating Pandas DataFrame for 712336 Arist Albums
	Shape --> (712336, 3)
DataFrame Shape is (712336, 3)
Current Time is Wed Jan 22, 2020 21:12:04 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 23 seconds.


,Known Albums,All Albums,Albums
1000,53,53,"{'44815': ['Archive One', 'Albums', True], '44..."
1000500,1,1,"{'1178875': ['Peak Controller', 'Miscellaneous..."
100200,9,9,"{'50511': ['PottpÃ¼ree', 'Albums', True], '219..."
1002000,3,3,"{'1002108': ['Are You Faithful?', 'Instruments..."
1002600,1,1,"{'1180905': ['Quiero', 'Vocals', True]}"


# Joining Artist ID DataFrame

In [30]:
from masterdb import createMasterDB

In [31]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

Current Time is Wed Jan 22, 2020 21:12:04 for 
=================================== Creating Artist ID DB ===================================
Creating Pandas DataFrame for 712336 Arist IDs
  Joining Artist Metadata
	Shape --> (712336, 9)
  Joining Artist Albums
	Shape --> (712336, 12)
Current Time is Wed Jan 22, 2020 21:12:10 for Done with 
=================================== Creating Artist ID DB ===================================
Process [{0}] took 5 seconds.
Saving data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.
Saved data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.


In [ ]:
from random import random